In [ ]:
import pandas as pd
import math

### Load 
df = pd.read_csv("/home/mouren/Data/valid_exp/summary_count_exonhancers_bilan.tsv", sep="\t", header=0)
df = df[~df['Log2FC'].str.contains('#', na=False)] # Remove rows with no log2FC values 

#Log2FC of merged replicates and standard deviation
df["Log2FC"] = df["Log2FC"].str.replace(',', '.')
df["Input_STD"] = df["Input_STD"].str.replace(',', '.')

df["Input_STD"] = pd.to_numeric(df["Input_STD"])
df["Log2FC"] = pd.to_numeric(df["Log2FC"])

#Input normalisé mergé
df["CPM_AVO4Exonhmaxi4"] = df["CPM_AVO4Exonhmaxi4"].str.replace(',', '.') 
df["CPM_AVO4Exonhmaxi4"] = pd.to_numeric(df["CPM_AVO4Exonhmaxi4"])

#Input normalisé 
df["CPM_AVO1ExonhDNA1"] = df["CPM_AVO1ExonhDNA1"].str.replace(',', '.')
df["CPM_AVO2ExonhDNA2"] = df["CPM_AVO2ExonhDNA2"].str.replace(',', '.')
df["CPM_AVO3ExonhDNA3"] = df["CPM_AVO3ExonhDNA3"].str.replace(',', '.')

df["CPM_AVO1ExonhDNA1"] = pd.to_numeric(df["CPM_AVO1ExonhDNA1"])
df["CPM_AVO2ExonhDNA2"] = pd.to_numeric(df["CPM_AVO2ExonhDNA2"])
df["CPM_AVO3ExonhDNA3"] = pd.to_numeric(df["CPM_AVO3ExonhDNA3"])

#Output normalisé mergé
df["CPM_AVO7ExonhcDNA3"] = df["CPM_AVO7ExonhcDNA3"].str.replace(',', '.')
df["CPM_AVO8ExonhcDNA4"] = df["CPM_AVO8ExonhcDNA4"].str.replace(',', '.')
df["CPM_AVO9ExonhcDNA5"] = df["CPM_AVO9ExonhcDNA5"].str.replace(',', '.')

df["CPM_AVO7ExonhcDNA3"] = pd.to_numeric(df["CPM_AVO7ExonhcDNA3"])
df["CPM_AVO8ExonhcDNA4"] = pd.to_numeric(df["CPM_AVO8ExonhcDNA4"])
df["CPM_AVO9ExonhcDNA5"] = pd.to_numeric(df["CPM_AVO9ExonhcDNA5"])

## To filter
df["count_sequence_AVO4Exonhmaxi4"] = pd.to_numeric(df["count_sequence_AVO4Exonhmaxi4"])

# removes input count reads less than 500 and standard deviation above 20
filtered_df = df[(df['count_sequence_AVO4Exonhmaxi4'] >= 500) & (df['Input_STD'] <= 20) & (df['Category2'].isin(["EEK","EESNP","EEMULTISNP"]))]

In [ ]:
### Make a dic matching each snp sequence with its ref
filtered_df['Name'] = filtered_df['ID'].apply(lambda x: '_'.join(x.split('_')[:3]))

df_no_snp = filtered_df[(filtered_df['Category2'] == "EEK")]
df_snp = filtered_df[(filtered_df['Category2'] == "EESNP")]
df_multisnp = filtered_df[(filtered_df['Category2'] == "EEMULTISNP")]

list_snp = []
for index,row in df_snp.iterrows():
    #find sample without snp
    matching_rows = df_no_snp.loc[df_no_snp['Name'] == row['Name']]
 
    if matching_rows.empty: #row without snp has been cut off because of input standarad deviation or not enough reads
        continue
    
    list_snp.append([row["ID"],matching_rows["ID"].iloc[0]])

for index,row in df_multisnp.iterrows():
    matching_rows = df_no_snp.loc[df_no_snp['Name'] == row['Name']]

    if matching_rows.empty: #row without snp has been cut off because of input standarad deviation or not enough reads
        continue
    
    list_snp.append([row["ID"],matching_rows["ID"].iloc[0]])

df_tosave = pd.DataFrame(list_snp)

df_tosave.to_csv("/home/mouren/Data/valid_exp/stat/seq_snp_ref_id.tsv",sep="\t",header=False,index=False)

/tmp/ipykernel_1745069/2163807241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Name'] = filtered_df['ID'].apply(lambda x: '_'.join(x.split('_')[:3]))


In [9]:
### Compute SNP vs REF
import numpy as np 
from scipy import stats
import statsmodels.stats.multitest as smm

snp_all = {}
with open("/home/mouren/Data/valid_exp/stat/seq_snp_ref_id.tsv") as f:
    for line in f:
        ref_row = filtered_df[filtered_df['ID'] == line.strip().split()[1]]
        snp_row = filtered_df[filtered_df['ID'] == line.strip().split()[0]]

        ref_log2fc = []
        snp_log2fc = []

        #compute fold change for each replicate for ref sequence
        try:
            ref_log2fc.append(math.log2(ref_row['CPM_AVO7ExonhcDNA3']/ref_row['CPM_AVO4Exonhmaxi4']))
        except Exception:
            pass
        try:
            ref_log2fc.append(math.log2(ref_row['CPM_AVO8ExonhcDNA4']/ref_row['CPM_AVO4Exonhmaxi4']))
        except Exception:
            pass
        try:
            ref_log2fc.append(math.log2(ref_row['CPM_AVO9ExonhcDNA5']/ref_row['CPM_AVO4Exonhmaxi4']))
        except Exception:
            pass
        #compute fold change for each replicate for snp sequence
        try:
            snp_log2fc.append(math.log2(snp_row['CPM_AVO7ExonhcDNA3']/snp_row['CPM_AVO4Exonhmaxi4']))
        except Exception:
            pass
        try:
            snp_log2fc.append(math.log2(snp_row['CPM_AVO8ExonhcDNA4']/snp_row['CPM_AVO4Exonhmaxi4']))
        except Exception:
            pass
        try:
            snp_log2fc.append(math.log2(snp_row['CPM_AVO9ExonhcDNA5']/snp_row['CPM_AVO4Exonhmaxi4']))
        except Exception:
            pass
        
        if len(ref_log2fc) >1 and len(snp_log2fc) >1: #if ref and snp sequence have more than one value 
            # Calculate means and variances
            mean_ref = np.mean(ref_log2fc)
            mean_mut = np.mean(snp_log2fc)
            var_ref = np.var(ref_log2fc, ddof=1)  # ddof=1 ensures that the unbiased estimator (sample variance) is used, dividing by n−1n−1 rather than nn.
            var_mut = np.var(snp_log2fc, ddof=1)

            # Number of replicates
            n_ref = len(ref_log2fc)
            n_mut = len(snp_log2fc)

            # Wald test statistic
            wald_stat = (mean_ref - mean_mut)**2 / ((var_ref / n_ref) + (var_mut / n_mut))

            # Compute the p-value (2 degrees of freedom for a two-tailed test)
            p_value = 1 - stats.chi2.cdf(wald_stat, df=1) # gives the upper-tail probability, which is the p-value.

            snp_all[line.strip().split()[0]] = p_value

print("Number of sig before FDR in all", sum(1 for v in snp_all.values() if v < 0.05))

# FDR correction & Save
significant, corrected_pvalues, alphacSidak, alphacBonf = smm.multipletests(list(snp_all.values()), alpha=0.05, method='fdr_bh')
corrected_dict = dict(zip(snp_all.keys(), corrected_pvalues))

with open('/home/mouren/Data/valid_exp/stat/all_starr_snp_vs_ref_FDR_wald.tsv', 'w') as file:
    for key,val in corrected_dict.items():
        ee_name = "_".join(key.split("_")[:3])
        type = key.split("_")[-1]
        file.write(f'{key}\t{val}\t{ee_name}\t{type}\n')

#with same number of element in ref and alt, its the same as if we take just more than one value in each list 
#181 sig after FDR with alpha=0.01

/tmp/ipykernel_1745069/280444882.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ref_log2fc.append(math.log2(ref_row['CPM_AVO7ExonhcDNA3']/ref_row['CPM_AVO4Exonhmaxi4']))
/tmp/ipykernel_1745069/280444882.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ref_log2fc.append(math.log2(ref_row['CPM_AVO8ExonhcDNA4']/ref_row['CPM_AVO4Exonhmaxi4']))
/tmp/ipykernel_1745069/280444882.py:25: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ref_log2fc.append(math.log2(ref_row['CPM_AVO9ExonhcDNA5']/ref_row['CPM_AVO4Exonhmaxi4']))
/tmp/ipykernel_1745069/280444882.py:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) 

Number of sig before FDR in all 654


In [6]:
### Make a dic matching each single snp sequence with its multi ref
filtered_df['Name'] = filtered_df['ID'].apply(lambda x: '_'.join(x.split('_')[:3]))

df_snp = filtered_df[(filtered_df['Category2'] == "EESNP")]
df_multisnp = filtered_df[(filtered_df['Category2'] == "EEMULTISNP")]

list_snp = []
for index,row in df_snp.iterrows():
    #find sample multi snp
    matching_rows = df_multisnp.loc[df_multisnp['Name'] == row['Name']]
 
    if matching_rows.empty: #row with multi snp don't exist or has been cut off because of input standarad deviation or not enough reads
        continue
    
    list_snp.append([row["ID"],matching_rows["ID"].iloc[0]])

df_tosave = pd.DataFrame(list_snp)
df_tosave.to_csv("/home/mouren/Data/valid_exp/stat/seq_EESNP_MULTISEQ_id.tsv",sep="\t",header=False,index=False)

/tmp/ipykernel_1745069/3137204533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Name'] = filtered_df['ID'].apply(lambda x: '_'.join(x.split('_')[:3]))


In [8]:
### Make a dic matching each single snp sequence with its multi ref and its ref 
dic_ref = {}
with open("/home/mouren/Data/valid_exp/stat/seq_snp_ref_id.tsv") as f:
    for line in f:
        dic_ref[line.strip().split()[0]] = line.strip().split()[1]

filtered_df['Name'] = filtered_df['ID'].apply(lambda x: '_'.join(x.split('_')[:3]))

df_snp = filtered_df[(filtered_df['Category2'] == "EESNP")]
df_multisnp = filtered_df[(filtered_df['Category2'] == "EEMULTISNP")]

list_snp = []
for index,row in df_snp.iterrows():
    #find sample multi snp
    matching_rows = df_multisnp.loc[df_multisnp['Name'] == row['Name']]
 
    if matching_rows.empty: #row with multi snp don't exist or has been cut off because of input standarad deviation or not enough reads
        continue
    
    if row["ID"] in dic_ref:
        list_snp.append([row["ID"],matching_rows["ID"].iloc[0],dic_ref[row["ID"]]])
    else:
        continue

df_tosave = pd.DataFrame(list_snp)
df_tosave.to_csv("/home/mouren/Data/valid_exp/stat/seq_EESNP_MULTISEQ_REF_id.tsv",sep="\t",header=False,index=False)

/tmp/ipykernel_1749051/1340612877.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Name'] = filtered_df['ID'].apply(lambda x: '_'.join(x.split('_')[:3]))


In [12]:
### There is only one MULTISNP per REF, while there can be several EESNP of the same REF
# We test significance difference between MULTISNP and EESNP
# Compute EESNP vs MULTISNP
import numpy as np 
from scipy import stats
import statsmodels.stats.multitest as smm

snp_all = {}
with open("/home/mouren/Data/valid_exp/stat/seq_EESNP_MULTISEQ_id.tsv") as f:
    for line in f:
        multi_row = filtered_df[filtered_df['ID'] == line.strip().split()[1]]
        snp_row = filtered_df[filtered_df['ID'] == line.strip().split()[0]]

        multi_log2fc = []
        snp_log2fc = []

        #compute fold change for each replicate for ref sequence
        try:
            multi_log2fc.append(math.log2(multi_row['CPM_AVO7ExonhcDNA3']/multi_row['CPM_AVO4Exonhmaxi4']))
        except Exception:
            pass
        try:
            multi_log2fc.append(math.log2(multi_row['CPM_AVO8ExonhcDNA4']/multi_row['CPM_AVO4Exonhmaxi4']))
        except Exception:
            pass
        try:
            multi_log2fc.append(math.log2(multi_row['CPM_AVO9ExonhcDNA5']/multi_row['CPM_AVO4Exonhmaxi4']))
        except Exception:
            pass
        #compute fold change for each replicate for snp sequence
        try:
            snp_log2fc.append(math.log2(snp_row['CPM_AVO7ExonhcDNA3']/snp_row['CPM_AVO4Exonhmaxi4']))
        except Exception:
            pass
        try:
            snp_log2fc.append(math.log2(snp_row['CPM_AVO8ExonhcDNA4']/snp_row['CPM_AVO4Exonhmaxi4']))
        except Exception:
            pass
        try:
            snp_log2fc.append(math.log2(snp_row['CPM_AVO9ExonhcDNA5']/snp_row['CPM_AVO4Exonhmaxi4']))
        except Exception:
            pass
        
        if len(multi_log2fc) >1 and len(snp_log2fc) >1: #if ref and snp sequence have more than one value 
            # Calculate means and variances
            mean_multi = np.mean(multi_log2fc)
            mean_mut = np.mean(snp_log2fc)
            var_ref = np.var(multi_log2fc, ddof=1)  # ddof=1 ensures that the unbiased estimator (sample variance) is used, dividing by n−1n−1 rather than nn.
            var_mut = np.var(snp_log2fc, ddof=1)

            # Number of replicates
            n_ref = len(multi_log2fc)
            n_mut = len(snp_log2fc)

            # Wald test statistic
            wald_stat = (mean_multi - mean_mut)**2 / ((var_ref / n_ref) + (var_mut / n_mut))

            # Compute the p-value (2 degrees of freedom for a two-tailed test)
            p_value = 1 - stats.chi2.cdf(wald_stat, df=1) # gives the upper-tail probability, which is the p-value.

            snp_all[line.strip().split()[0]] = p_value

# FDR & Save
significant, corrected_pvalues, alphacSidak, alphacBonf = smm.multipletests(list(snp_all.values()), alpha=0.05, method='fdr_bh')
corrected_dict = dict(zip(snp_all.keys(), corrected_pvalues))

with open('/home/mouren/Data/valid_exp/stat/all_starr_EESNP_vs_MULTI_FDR_wald.tsv', 'w') as file:
    for key,val in corrected_dict.items():
        ee_name = "_".join(key.split("_")[:3])
        type = key.split("_")[-1]
        file.write(f'{key}\t{val}\t{ee_name}\t{type}\n')

/tmp/ipykernel_1745069/615952611.py:19: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  multi_log2fc.append(math.log2(multi_row['CPM_AVO7ExonhcDNA3']/multi_row['CPM_AVO4Exonhmaxi4']))
/tmp/ipykernel_1745069/615952611.py:23: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  multi_log2fc.append(math.log2(multi_row['CPM_AVO8ExonhcDNA4']/multi_row['CPM_AVO4Exonhmaxi4']))
/tmp/ipykernel_1745069/615952611.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  multi_log2fc.append(math.log2(multi_row['CPM_AVO9ExonhcDNA5']/multi_row['CPM_AVO4Exonhmaxi4']))
/tmp/ipykernel_1745069/615952611.py:32: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use f